In [22]:
# Marketing response prediction based on AdaBoost
import time 
import numpy as np  
import pandas as pd  
from sklearn.preprocessing import OneHotEncoder  
from sklearn.model_selection import StratifiedKFold, cross_val_score  
from sklearn.feature_selection import SelectPercentile, f_classif  
from sklearn.ensemble import AdaBoostClassifier  
from sklearn.pipeline import Pipeline  
from sklearn.metrics import accuracy_score  


In [23]:
data = pd.read_excel('../../data/order.xlsx', sheetname=0) 

/usr/local/lib/python3.5/dist-packages/pandas/util/_decorators.py:118: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)


In [24]:
X = data.drop('response', axis=1)
y = data['response']

In [25]:
data.head()

,age,total_pageviews,edu,edu_ages,user_level,industry,value_level,act_level,sex,blue_money,red_money,work_hours,region,response
0,39.0,77516.0,1.0,13.0,1.0,1.0,1,1.0,1.0,2174,0.0,40,1.0,0
1,50.0,83311.0,1.0,13.0,2.0,2.0,2,1.0,1.0,0,0.0,13,1.0,0
2,38.0,215646.0,2.0,9.0,3.0,3.0,1,1.0,1.0,0,0.0,40,1.0,0
3,53.0,234721.0,2.0,7.0,2.0,3.0,2,2.0,1.0,0,0.0,40,1.0,0
4,28.0,338409.0,1.0,13.0,2.0,4.0,3,2.0,0.0,0,0.0,40,2.0,0


In [26]:
data.shape

(39999, 14)

In [27]:
data.dtypes

age                float64
total_pageviews    float64
edu                float64
edu_ages           float64
user_level         float64
industry           float64
value_level          int64
act_level          float64
sex                float64
blue_money           int64
red_money          float64
work_hours           int64
region             float64
response             int64
dtype: object

In [28]:
data.describe()

,age,total_pageviews,edu,edu_ages,user_level,industry,value_level,act_level,sex,blue_money,red_money,work_hours,region,response
count,39998.000000,3.999800e+04,39998.000000,39998.000000,39998.000000,39997.000000,39999.000000,39998.000000,39998.000000,39999.000000,39998.000000,39999.000000,39997.000000,39999.000000
mean,38.589654,1.895136e+05,2.511626,10.076754,2.087004,5.677126,2.546289,1.221036,0.668083,1089.142529,87.379394,40.442486,2.251519,0.239606
std,13.663490,1.053109e+05,1.638110,2.573384,1.260992,3.395948,1.443210,0.626618,0.470907,7491.275548,402.930350,12.376033,4.913482,0.426848
min,17.000000,1.228500e+04,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000
25%,28.000000,1.175282e+05,2.000000,9.000000,1.000000,3.000000,1.000000,1.000000,0.000000,0.000000,0.000000,40.000000,1.000000,0.000000
50%,37.000000,1.783410e+05,2.000000,10.000000,2.000000,5.000000,2.000000,1.000000,1.000000,0.000000,0.000000,40.000000,1.000000,0.000000
75%,48.000000,2.372685e+05,2.000000,12.000000,2.000000,8.000000,4.000000,1.000000,1.000000,0.000000,0.000000,45.000000,1.000000,0.000000
max,90.000000,1.484705e+06,10.000000,16.000000,7.000000,15.000000,6.000000,5.000000,1.000000,99999.000000,4356.000000,99.000000,41.000000,1.000000


In [29]:
data.isnull().sum()

age                1
total_pageviews    1
edu                1
edu_ages           1
user_level         1
industry           2
value_level        0
act_level          1
sex                1
blue_money         0
red_money          1
work_hours         0
region             2
response           0
dtype: int64

In [30]:
data.response.value_counts()

0    30415
1     9584
Name: response, dtype: int64

In [31]:
data.dtypes

age                float64
total_pageviews    float64
edu                float64
edu_ages           float64
user_level         float64
industry           float64
value_level          int64
act_level          float64
sex                float64
blue_money           int64
red_money          float64
work_hours           int64
region             float64
response             int64
dtype: object

In [35]:
# fill null values
na_rules = {'age': data['age'].mean(),
            'total_pageviews': data['total_pageviews'].mean(),
            'edu': data['edu'].median(),
            'edu_ages': data['edu_ages'].median(),
            'user_level': data['user_level'].median(),
            'industry': data['user_level'].median(),
            'act_level': data['act_level'].median(),
            'sex': data['sex'].median(),
            'red_money': data['red_money'].mean(),
            'region': data['region'].median()
            } 
X = X.fillna(na_rules) 

In [36]:
# datatype transformation
var_list = {'edu': 'int32',
            'user_level': 'int32',
            'industry': 'int32',
            'value_level': 'int32',
            'act_level': 'int32',
            'sex': 'int32',
            'region': 'int32'
            }  
for var, type in var_list.items():  
    X[var] = X[var].astype(type)

In [37]:
X.isnull().sum()

age                0
total_pageviews    0
edu                0
edu_ages           0
user_level         0
industry           0
value_level        0
act_level          0
sex                0
blue_money         0
red_money          0
work_hours         0
region             0
dtype: int64

In [17]:
def symbol_con(df, enc_object=None, train=True):
    # one hot encoding
    convert_cols = ['edu', 'user_level', 'industry', 'value_level', 'act_level', 'sex', 'region']  
    df_con = df[convert_cols] 
    df_org = df[['age', 'total_pageviews', 'edu_ages', 'blue_money', 'red_money', 'work_hours']].values
    if train == True:  
        enc = OneHotEncoder()  
        enc.fit(df_con)  
        df_con_new = enc.transform(df_con).toarray()  
        new_matrix = np.hstack((df_con_new, df_org))  
        return new_matrix, enc
    else:
        df_con_new = enc_object.transform(df_con).toarray()  
        new_matrix = np.hstack((df_con_new, df_org))
        return new_matrix


In [38]:
X_new, enc = symbol_con(X, enc_object=None, train=True)

In [41]:
X_new[0]

array([1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00,
       1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e

In [21]:
def get_best_model(X, y):
    '''
    use crossvalidation select parameter for the model
    '''
    transform = SelectPercentile(f_classif, percentile=50)  
    model_adaboost = AdaBoostClassifier()  
    model_pipe = Pipeline(steps=[('ANOVA', transform), ('model_adaboost', model_adaboost)])  
    cv = StratifiedKFold(5)  
    n_estimators = [20, 50, 80, 100]  
    score_methods = ['accuracy', 'f1', 'precision', 'recall', 'roc_auc']  
    mean_list = list()  
    std_list = list() 
    for parameter in n_estimators:  
        t1 = time.time() 
        score_list = list()  
        print ('set parameters: %s' % parameter) 
        for score_method in score_methods:  
            model_pipe.set_params(model_adaboost__n_estimators=parameter)  
            score_tmp = cross_val_score(model_pipe, X, y, scoring=score_method, cv=cv)  
            score_list.append(score_tmp)  
        score_matrix = pd.DataFrame(np.array(score_list), index=score_methods)  
        score_mean = score_matrix.mean(axis=1).rename('mean')  
        score_std = score_matrix.std(axis=1).rename('std')  
        score_pd = pd.concat([score_matrix, score_mean, score_std], axis=1) 
        mean_list.append(score_mean) 
        std_list.append(score_std)  
        print (score_pd.round(2))  
        print ('-' * 60)
        t2 = time.time()  
        tt = t2 - t1  
        print ('time: %s' % str(tt))  
    mean_matrix = np.array(mean_list).T  
    std_matrix = np.array(std_list).T  
    mean_pd = pd.DataFrame(mean_matrix, index=score_methods, columns=n_estimators)  
    std_pd = pd.DataFrame(std_matrix, index=score_methods, columns=n_estimators) 
    print ('Mean values for each parameter:')
    print (mean_pd)  
    print ('Std values for each parameter:')
    print (std_pd)  
    print ('-' * 60)
    return transform

In [42]:
model = get_best_model(X_new, y)


set parameters: 20


/usr/local/lib/python3.5/dist-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [85] are constant.
  UserWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
/usr/local/lib/python3.5/dist-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [85] are constant.
  UserWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
/usr/local/lib/python3.5/dist-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [85] are constant.
  UserWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
/usr/local/lib/python3.5/dist-packages/sklearn/fe

              0     1     2     3     4  mean   std
accuracy   0.85  0.85  0.86  0.86  0.86  0.85  0.01
f1         0.66  0.64  0.66  0.66  0.66  0.66  0.01
precision  0.72  0.74  0.76  0.78  0.77  0.75  0.02
recall     0.60  0.57  0.59  0.58  0.57  0.58  0.01
roc_auc    0.91  0.90  0.91  0.91  0.91  0.91  0.00
------------------------------------------------------------
time: 10.778498411178589
set parameters: 50


/usr/local/lib/python3.5/dist-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [85] are constant.
  UserWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
/usr/local/lib/python3.5/dist-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [85] are constant.
  UserWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
/usr/local/lib/python3.5/dist-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [85] are constant.
  UserWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
/usr/local/lib/python3.5/dist-packages/sklearn/fe

              0     1     2     3     4  mean   std
accuracy   0.86  0.86  0.86  0.87  0.86  0.86  0.00
f1         0.66  0.66  0.67  0.69  0.68  0.67  0.01
precision  0.75  0.76  0.77  0.78  0.77  0.76  0.01
recall     0.59  0.58  0.59  0.62  0.61  0.60  0.02
roc_auc    0.91  0.91  0.91  0.92  0.92  0.91  0.00
------------------------------------------------------------
time: 23.52108097076416
set parameters: 80


/usr/local/lib/python3.5/dist-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [85] are constant.
  UserWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
/usr/local/lib/python3.5/dist-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [85] are constant.
  UserWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
/usr/local/lib/python3.5/dist-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [85] are constant.
  UserWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
/usr/local/lib/python3.5/dist-packages/sklearn/fe

              0     1     2     3     4  mean   std
accuracy   0.86  0.86  0.86  0.87  0.86  0.86  0.00
f1         0.67  0.67  0.68  0.70  0.68  0.68  0.01
precision  0.76  0.77  0.77  0.79  0.76  0.77  0.01
recall     0.60  0.59  0.61  0.62  0.62  0.61  0.01
roc_auc    0.92  0.92  0.92  0.92  0.92  0.92  0.00
------------------------------------------------------------
time: 40.48507523536682
set parameters: 100


/usr/local/lib/python3.5/dist-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [85] are constant.
  UserWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
/usr/local/lib/python3.5/dist-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [85] are constant.
  UserWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
/usr/local/lib/python3.5/dist-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [85] are constant.
  UserWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
/usr/local/lib/python3.5/dist-packages/sklearn/fe

              0     1     2     3     4  mean   std
accuracy   0.86  0.86  0.87  0.87  0.86  0.86  0.00
f1         0.67  0.67  0.69  0.70  0.69  0.68  0.01
precision  0.76  0.77  0.78  0.79  0.76  0.77  0.01
recall     0.60  0.60  0.61  0.63  0.62  0.61  0.01
roc_auc    0.92  0.92  0.92  0.92  0.92  0.92  0.00
------------------------------------------------------------
time: 42.5383358001709
Mean values for each parameter:
                20        50        80        100
accuracy   0.853946  0.859922  0.862647  0.863672
f1         0.656329  0.672173  0.679723  0.682772
precision  0.753126  0.764994  0.770094  0.771560
recall     0.582011  0.599542  0.608411  0.612376
roc_auc    0.908324  0.914994  0.918613  0.919941
Std values for each parameter:
                20        50        80        100
accuracy   0.005260  0.004636  0.004463  0.004834
f1         0.009512  0.012836  0.011527  0.011929
precision  0.023295  0.010208  0.011250  0.012753
recall     0.013767  0.016128  0.013591  

In [43]:
model

SelectPercentile(percentile=50,
         score_func=<function f_classif at 0x7f332688b158>)

In [46]:
model.fit(X_new, y) 
X_final = model.transform(X_new) 
final_model = AdaBoostClassifier(n_estimators=100)
final_model.fit(X_final, y)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=100, random_state=None)

(39999, 46)